# Tokenização
Este notebook tem como objetivo aplicar vetorização e tokenização de enunciados e alternativas por meio de embeddings de palavras. O processo inclui etapas de pré-processamento textual, geração dos vetores e organização dos resultados para uso posterior em análises ou em modelos de aprendizado de máquina.

## Apoio

In [1]:
# Importando bibliotecas
import pandas as pd
import numpy as np
import re
from gensim.models import KeyedVectors

In [39]:
# Leitura do dataset
df_enem = pd.read_csv('../data/cleaned_data.csv')
df_enem.head(10)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...
5,6,A,2.04543,0.74139,0.20814,2017,Descrição da fotografia: É retratada uma mulhe...,A: ambiguidade do trabalho doméstico exercido ...,ambiguidade do trabalho doméstico exercido pel...,integração dos escravos aos valores das classe...,adornada ama ambos apoiadas augusto branco bra...,A: ama ambiguidade desenvolvendo doméstico exe...,ama ambiguidade desenvolvendo doméstico exerci...,associando cativos centralizando classes condi...
6,7,D,4.02261,0.96775,0.17554,2017,A conﬁguração do espaço urbano da região do En...,A: expansão vertical; B: polarização naciona...,segregação socioespacial,expansão vertical; polarização nacional.; eman...,acesso acumulação aglomerado aglomerações asse...,A: expansão vertical; B: nacional polarização;...,segregação socioespacial,comercial desregulamentação emancipação expans...
7,8,B,1.33884,2.80117,0.16665,2017,"México, Colômbia, Peru e Chile decidiram segui...",A: promover a livre circulação de trabalhadore...,fomentar a competitividade no mercado externo,promover a livre circulação de trabalhadores; ...,acordo aduaneiras aliança caminho cento chile ...,A: circulação livre promover trabalhadores; B:...,competitividade externo fomentar mercado,adotar agrícola alfandegária cambiais circulaç...
8,9,E,3.15884,0.92459,0.09500,2017,Descrição da tabela: Indica a tipologia de áre...,A: erosão laminar; B: intemperismo físico; C...,recarga dos aquíferos,erosão laminar; intemperismo físico; enchente ...,agrícola apresentando bacias cento chuva cober...,A: erosão laminar; B: físico intemperismo; C: ...,aquíferos recarga,cidades compactação enchente erosão físico int...
9,10,A,0.67598,0.55190,0.00798,2017,O desgaste acelerado sempre existirá se o agri...,A: Elevação da acidez; B: Ampliação da sali

## Word Embedding com Word2Vec

In [38]:
# Carregando o modelo de embedding com 300 dimensões
model_300 = KeyedVectors.load_word2vec_format("/home/alesouza/USP/9_semestre/pcd-iii/projeto_enem/cbow_s300.txt")

In [37]:
# Carregando o modelo de embedding com 50 dimensões
model_50 = KeyedVectors.load_word2vec_format("/home/alesouza/USP/9_semestre/pcd-iii/projeto_enem/cbow_s50.txt")

In [61]:
# Carregando o modelo de embedding com 100 dimensões
model_100 = KeyedVectors.load_word2vec_format("/home/alesouza/USP/9_semestre/pcd-iii/projeto_enem/cbow_s100.txt")

In [62]:
# Dicionário auxiliar que armazena palavras já encontradas
vetor_cache = {}

# Função auxiliar que irá retornar os vetores de palavras
def generate_word2vec_embeddings(data, model):
  not_found_words = set()
  embeddings = []

  for item in data:
    word_vectors = []
    words = re.sub(r'[^\w\s]', '', item.lower()).split()

    for word in words:
      try:
        if word in vetor_cache:
          word_vectors.append(vetor_cache[word])
        else:
          vetor_cache[word] = model[word]
          word_vectors.append(model[word])
      except KeyError:
        not_found_words.add(word)

    if word_vectors:
          # Calcula a média dos vetores
          mean_vector = np.mean(word_vectors, axis=0)
          embeddings.append(mean_vector)
    else:
          # Se não encontrou nenhuma palavra, adiciona um vetor nulo
          embeddings.append(np.zeros(model.vector_size))

  return embeddings, not_found_words


### Vetorização dos enunciados

#### 300 dimensões

In [40]:
df_enem_300 = df_enem.copy()

In [41]:
df_enem_300["enunciado_embeddings_word2vec"], not_found_words_enunciado = (
    generate_word2vec_embeddings(df_enem_300["enunciado_limpo"], model_300)
)

In [42]:
len(not_found_words_enunciado)

191

#### 50 dimensões

In [43]:
df_enem_50 = df_enem.copy()

In [47]:
df_enem_50["enunciado_embeddings_word2vec"], not_found_words_enunciado = (
    generate_word2vec_embeddings(df_enem_50["enunciado_limpo"], model_50)
)

In [48]:
len(not_found_words_enunciado)

191

#### 100 dimensões

In [63]:
df_enem_100 = df_enem.copy()

In [64]:
df_enem_100["enunciado_embeddings_word2vec"], not_found_words_enunciado = (
    generate_word2vec_embeddings(df_enem_100["enunciado_limpo"], model_100)
)

In [65]:
len(not_found_words_enunciado)

191

### Vetorização das alternativas corretas

#### 300 dimensões

In [53]:
df_enem_300["gabarito_embeddings_word2vec"], not_found_words_gabarito = (
    generate_word2vec_embeddings(df_enem_300["gabarito_texto_limpo"].astype(str), model_300)
)

In [54]:
len(not_found_words_gabarito)

5

In [55]:
not_found_words_gabarito

{'biopolíticos',
 'conduzireis',
 'extrapeninsulares',
 'multiescalares',
 'socioafetivos'}

#### 50 dimensões

In [49]:
df_enem_50["gabarito_embeddings_word2vec"], not_found_words_gabarito = (
    generate_word2vec_embeddings(df_enem_50["gabarito_texto_limpo"].astype(str), model_50)
)

In [50]:
len(not_found_words_gabarito)

5

In [51]:
not_found_words_gabarito

{'biopolíticos',
 'conduzireis',
 'extrapeninsulares',
 'multiescalares',
 'socioafetivos'}

#### 100 dimensões

In [66]:
df_enem_100["gabarito_embeddings_word2vec"], not_found_words_gabarito = (
    generate_word2vec_embeddings(df_enem_100["gabarito_texto_limpo"].astype(str), model_100)
)

In [67]:
len(not_found_words_gabarito)

5

In [56]:
df_enem_300.head(5)

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,"[-0.017395772, -0.004402813, -0.046908338, 0.0...","[-0.16301225, -0.142375, 0.012171499, -0.00457..."
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,"[-0.051949076, 0.03128451, -0.072241075, 0.028...","[-0.18672037, 0.04755275, -0.17406088, -0.0398..."
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,"[0.026049094, 0.043251127, -0.029757027, -0.00...","[0.011740998, 0.07592634, -0.016144669, 0.1384..."
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,"[-0.005251272, 0.039871164, -0.018038195, 0.06...","[-0.073669255, 0.12609875, 0.063628, 0.1261957..."
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,"[0.01738678, 0.0033733728, -0.027247325, 0.002...","[0.11709199, -0.16062833, 0.082869664, -0.1195..."


In [59]:
len(df_enem_300['gabarito_embeddings_word2vec'].iloc[0])

300

In [58]:
len(df_enem_50['gabarito_embeddings_word2vec'].iloc[0])

50

In [68]:
len(df_enem_100['gabarito_embeddings_word2vec'].iloc[0])

100

## Salvando os datasets

In [69]:
df_enem_300.to_pickle('../data/final/enem_data_embeddings_300.pkl')
df_enem_50.to_pickle('../data/final/enem_data_embeddings_50.pkl')
df_enem_100.to_pickle('../data/final/enem_data_embeddings_100.pkl')